
**Sample ID:** CES_synthetic_account_management_2.2_conversation_79_12

**Query:** No, I've changed my mind.

**DB Type:** Base Case

**Case Description:**

**Global/Context Variables:**

**Datetime Context Variables:**

**APIs:**
- ces_account_management

**Databases:**

# Set Up

## Download relevant files

In [ ]:
import io
import os
import sys
import zipfile
import shutil
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

VERSION = "0.1.4_ces_account_management"  # Pass the version of the API
CONTENT_DIR = '/content'
APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')

APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'
ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/', 'Schemas/']

# Cleanup
for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Auth
auth.authenticate_user()
drive_service = build('drive', 'v3')

def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(output_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if show_progress:
                print(f"Download progress: {int(status.progress() * 100)}%")

print(f"Searching for APIs zip file with version {VERSION} in folder: {APIS_FOLDER_ID}...")
apis_file_id = None
try:
    query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    for file in results.get('files', []):
        if file['name'].lower() == f'apis_v{VERSION.lower()}.zip':
            apis_file_id = file['id']
            print(f"Found: {file['name']} (ID: {apis_file_id})")
            break
except Exception as e:
    print(f"Error listing files: {e}")

if not apis_file_id:
    sys.exit(f"❌ APIs zip V{VERSION} not found.")

print(f"Downloading APIs zip {apis_file_id}...")
download_drive_file(drive_service, apis_file_id, ZIP_PATH)

print(f"Extracting {ZIP_PATH}...")
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    for member in zip_ref.namelist():
        if any(member.startswith(p) for p in ITEMS_TO_EXTRACT):
            zip_ref.extract(member, CONTENT_DIR)

os.remove(ZIP_PATH)

if os.path.exists(APIS_DIR):
    sys.path.append(APIS_DIR)

for p in [APIS_DIR, DBS_DIR, SCRIPTS_DIR]:
    print(f"{'✅' if os.path.exists(p) else '❌'} {p}")

## Install Dependencies and Clone Repositories

In [ ]:

!pip install uv
!uv pip install -r /content/APIs/requirements.txt


# Import APIs and initiate DBs

In [ ]:
# import APIs and initialize DBs
import ces_account_management

ces_account_management.load_state('./DBs/CesAccountManagementDefaultDB.json')

ces_account_management.get_customer_account_details(**{'accountId': '7135550145'})
ces_account_management.query_available_plans_and_features(**{'query': 'I need a plan with more data'})

# Initial Assertion
1. Assert that there exists an account details record with the provided identifier. 
2. Assert that the customer name associated with the provided identifier matches the name confirmed by the user.
3. Assert that the conversation hasn't been cancelled.
* Use all the assertions in previous turns + final assertion of the immediate previous turn (turn n-2)        

In [ ]:
import ces_account_management
from Scripts.assertions_utils import *

# Set error mode to dict
from common_utils import get_error_manager

error_manager = get_error_manager()
error_manager.set_error_mode("error_dict")

# local Variable
accountId = ''
# phone_no = '' # if phone number is given in place of account ID
customerName = ''

# Get account detail from accountId provided
account_details = ces_account_management.get_customer_account_details(accountId=accountId)

# 1.  Assert that there exists an account details record with the provided identifier.
assert not account_details.get('exceptionType'), f'Account ID : {accountId} does not exist.' # for account ID
# assert not account_details.get('exceptionType'), f'Unable to find account details with the provided phone number: {phone_no}.' # for phone number

# 2. Assert that the customer name associated with the provided identifier matches the name confirmed by the user.
assert compare_strings(account_details.get('customerName'), customerName), f'Customer name : {customerName} does not match.'

# Get the conversation response
response  = ces_account_management.utils.get_end_of_conversation_status()

# Get the end of conversation status
end_conv_status = response.get('cancel', None)

# 3. Assert that the call isn't canceled.
assert end_conv_status is None, 'The call has been canceled.'


# Action

In [ ]:
#proto_ignore
import ces_account_management

In [ ]:
ces_account_management.cancel(**{'reason': 'user wants to cancel.'})

# Golden Answer
Okay, I have canceled this request. Thank you for calling.

# Final Assertion
1. Assert that the call has been cancelled.
2. Assert that the reason for cancel call is provided.
3. Assert that the status of cancel call is provided.

In [ ]:
import ces_account_management
from Scripts.assertions_utils import *

response  = ces_account_management.utils.get_end_of_conversation_status()

# Get the end of conversation status
end_conv_status = response.get('cancel', None)

# 1. Assert that the call has been cancelled
assert end_conv_status is not None, 'The call has not been cancelled.'

# 2. Assert that the reason for cancel call is provided
assert end_conv_status.get('reason') is not None or not compare_strings(end_conv_status.get('reason',''),''), 'Missing cancel call reason.'

# 3. Assert that the status for cancel call is provided
assert end_conv_status.get('status') is not None or not compare_strings(end_conv_status.get('status',''),''), 'Missing cancel call status.'